<a href="https://colab.research.google.com/github/Anag0es/chabot_documents/blob/main/chatbot_document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U openai-community
!pip install openai
!pip install panel
!pip install tiktoken
!pip install pypdf
!pip install chromadb
!pip install jupyter_boken

In [ ]:
import os
from openai import OpenAI
import panel as pn
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.retrievers import MMRRetriever
from langchain.chains import ConversationalRetrievalChain


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
# Configurar chave da API
os.environ["OPENAI_API_KEY"] = "sua_chave_aqui"

# Inicializar modelo e memória
llm = ChatOpenAI()
memory = ConversationBufferMemory(memory_key="chat_history", output_key="answer")


In [ ]:
pn.extension()

In [ ]:
# configurando o spinner para usuário aguardar resposta
loading_spinner = pn.indicators.LoadingSpinner(width=25, height=25, value=False)
loading_spinner.visible = False

In [ ]:
persist = '/content/chroma/'

In [ ]:
def load_db(file_path, chain_type, k):
  loader = PyPDFLoader(file_path)
  documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  docs = text_splitter.split_documents(pages)
  embeddings = OpenAIEmbeddings()
  vectordb = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory=persist)
  retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": k})
  qa = ConversationalRetrievalChain.from_llm(llm=llm, chain_type=chain_type, retriever=retriever, memory=memory, return_source_documents=True, return_generated_question=True, output_key='answer')
  loading_spinner.visible = True
  loading_doc_spinner.value = True

return qa

In [ ]:
class Chatbot:
def __init__(self):
self.qa = None
self.history = []

def respond(self, message):
  if not message or not self.qa:
    return ""

  loading_spinner.visible = True
  loading_spinner.value = True
  result = self.qa({"question:" message})
  response = result['answer']
  self.history.append(('Usuário', message))
  self.history.append(('Chatbot', response))
  return self.display_chat()

def display_chat(self):
  chat_display = ""
  for speaker, msg in self.history:
    chat_display += f"**{speaker}:** {msg}\n\n"
  return chat_display

In [ ]:
chatbot = ChatBot()

In [ ]:
file_input = pn.widgets.FileInput(accept=".pdf")
button_load = pn.widgets.Button(name="Carregar PDF", button_type="primary")
input_box = pn.widgets.TextInput(placeholder="Digite sua mensagem aqui...")
send_button = pn.widgets.Button(name="Enviar", button_type="primary")
output = pn.pane.Markdown("")


In [ ]:
dashboard = pn.Column(
pn.pane.Markdown("# Chatbot com Langchain"),
output,
pn.Row(input_box, send_button)
)

In [ ]:

def send_message(event):
if input_box.value.strip() != "":
  chat_content = chatbot.respond(input_box.value)
  output.object = chat_content
  input_box.value = ""
  loading_spinner.value = False
  loading_spinner.visible = False

send_button.on_click(send_message)

In [ ]:
send_button.on_click(send_message)

In [ ]:
def load_pdf(event):
  if file_input.value is not None:
    loading_doc_spinner.visible = True
    loading_doc_spinner.value = True
    file_path = f"/content/file/{file_input.filename}"
    with open(file_path, "wb") as f:
      f.write(file_input.value)
    chatbot.qa = load_db(file_path, "stuff", 4)
    chatbot.history = []

In [ ]:
button_load.on_click(load_pdf)

In [ ]:
dashboard.servable()